# Neopets Automation
Patrick Vo

Program that automatically collects neopets freebies and whatnot

In [2]:
#Import the necessary libraries
import pyautogui as pag
pag.FAILSAFE = True
pag.PAUSE = 0.10

import pandas as pd
import time
import webbrowser

## The Special Ones
This section is comprised of Neopets dailies that require multiple clicks or other unusual instructions. Because of this, they can't be put into a single function.

In [65]:
def wise_old_king():
    webbrowser.open('http://www.neopets.com/medieval/wiseking.phtml')
    time.sleep(4)
    
    # Clears out any events that might pop up on the page
    pag.moveTo(1786,706)
    pag.click()
    time.sleep(2)
    
    #Move the cursor into position
    pag.moveTo(778, 604)
    pag.click()
    
    for i in range (0,7):
        pag.typewrite(['down', 'down', 'down', 'tab'])
    
    pag.moveTo(1027,675)
    pag.click()
    pag.hotkey('ctrl', 'w')
    
    
def grumpy_old_king():
    webbrowser.open('http://www.neopets.com/medieval/grumpyking.phtml')
    time.sleep(4)
    pag.moveTo(1786,706)
    pag.click()
    time.sleep(2)
    pag.moveTo(721, 664)
    pag.click()
    
    for i in range (0,18):
        pag.typewrite(['down', 'down', 'down', 'tab'])
    
    pag.moveTo(924,860)
    pag.click()
    pag.hotkey('ctrl', 'w')
    
def lost_tomb():
    webbrowser.open('http://www.neopets.com/worlds/geraptiku/tomb.phtml')
    time.sleep(4)
    pag.moveTo(1786,706)
    pag.click()
    time.sleep(2)
    pag.moveTo(848, 761)
    pag.click()
    time.sleep(4)
    pag.moveTo(1009,716)
    pag.click()
    time.sleep(4)
    pag.hotkey('ctrl', 'w')
    
def qasalan_expellibox():
    def neopian_one_clicks():
    webbrowser.open('http://ncmall.neopets.com/mall/shop.phtml?page=giveaway')
    time.sleep(4)
    pag.moveTo(1786,706)
    pag.click()
    time.sleep(2)
    pag.moveTo(1127,846)
    pag.click()
    time.sleep(2)
    pag.click()
    pag.hotkey('ctrl', 'w')

## The One-Clicks
This section is made up of dailies that can activated by a single click. The script uses pandas to read in a CSV file, then passes in the position and URL of the page. The function proceeds through, clicking one-by-one. Note that some of the dailies, such as Coltzan's Shrine, can actually be done multiple times a day, following a cooldown period. These have 'time_delay' parameters and will be called after the cooldown period has elapsed.

In [106]:
def neopian_one_clicks(x_pos, y_pos, url):
    webbrowser.open(url)
    time.sleep(4)
    
    # Moves the cursor and clicks nowhere in particular. This gets rid of any unusual overlays
    pag.moveTo(1786,706)
    pag.click()
    time.sleep(2)
    
    pag.moveTo(x_pos,y_pos)
    pag.click()
    time.sleep(3)
    pag.hotkey('ctrl', 'w')
    

# The main function

In [ ]:
# The main function calls the odd clicks first, then loads in the actions csv. It passes the rows
# of the actions csv into the neopian_one_clicks function
def main():
    wise_old_king()
    grumpy_old_king()
    qasalan_expellibox()
    lost_tomb()
    
    # Read in the actions csv 
    one_clicks = pd.read_csv('one_clicks.csv')
    one_clicks['timestamp'] = -1
    
    # For every row, pass in the thing
    for i in range(0, len(one_clicks)):
        
        actions = neopian_one_clicks.iloc[i]
        neopian_one_clicks(actions['X_position'], actions['Y_position'], actions['URL'])
        
        # If the Time_delay variable isn't negative 1, then change the timestamp variable
        if actions['Time_Delay'] > 0:
            neopian_one_clicks.iloc[i]['timestamp'] = time()
    
    # For all the actions that can be done multiple times a day, check every 15 minutes
    #     to see if the timer has reset yet
    repeatables = one_clicks[one_clicks['Time_Delay'] > 0]
    while (True):
        time.sleep(900)
        for i in range (0, len(one_clicks)):
            maybe =  one_clicks.iloc[i]
            
            # If the time difference in seconds is larger than the reset time in seconds
            #    then do the daily again
            if ((time() - maybe['timestamp']) > (maybe['Time_Delay'] * 3600)):
                neopian_one_clicks(maybe['X_position'], maybe['Y_position'], maybe['URL'])
                maybe['timestamp'] = time()
                
                

(956, 807)